In [13]:
import os, sys
import cv2, glob, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import warnings
warnings.filterwarnings('ignore')


# from ultralytics import YOLO
# model = YOLO("yolov8n.pt")
# results = model.train(data="yolo-dataset/yolo.yaml", epochs=2, imgsz=1080, batch=16)

from ultralytics import YOLOv10

# model = YOLOv10()
# If you want to finetune the model with pretrained weights, you could load the 
# pretrained weights like below
# model = YOLOv10.from_pretrained('jameslahm/yolov10{n/s/m/b/l/x}')
# or
# wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10{n/s/m/b/l/x}.pt
model = YOLOv10('yolov10x.pt')

model.train(data='yolo-dataset/yolo.yaml', epochs=500, batch=4, imgsz=640)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov10x.pt, data=yolo-dataset/yolo.yaml, epochs=500, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

train: Scanning D:\python\kaggle\2024“大运河杯”数据开发应用创新大赛——城市治理\AI_Camp5_baseline_CV\yolo-dataset\train... 58732 images, 5 backgrounds, 0 corrupt: 100%|██████████| 58737/58737 [00:23<00:00, 2452.91it/s]


train: New cache created: D:\python\kaggle\2024“大运河杯”数据开发应用创新大赛——城市治理\AI_Camp5_baseline_CV\yolo-dataset\train.cache


val: Scanning D:\python\kaggle\2024“大运河杯”数据开发应用创新大赛——城市治理\AI_Camp5_baseline_CV\yolo-dataset\val... 13602 images, 3 backgrounds, 0 corrupt: 100%|██████████| 13605/13605 [00:05<00:00, 2474.26it/s]


val: New cache created: D:\python\kaggle\2024“大运河杯”数据开发应用创新大赛——城市治理\AI_Camp5_baseline_CV\yolo-dataset\val.cache
Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 185 weight(decay=0.0), 198 weight(decay=0.0005), 197 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train3
Starting training for 500 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


      1/500      5.46G      1.649       2.47      1.735      1.681      3.767      1.656         30        640:   3%|▎         | 374/14685 [03:19<2:06:55,  1.88it/s]


KeyboardInterrupt: 

In [ ]:
category_labels = ["非机动车违停", "机动车违停", "垃圾桶满溢", "违法经营"]

if not os.path.exists('result/'):
    os.mkdir('result')

In [ ]:
from ultralytics import YOLOv10
model = YOLOv10("runs/detect/train3/weights/best.pt")
import glob

for path in glob.glob('../data/测试集/*.mp4'):
    submit_json = []
    results = model(path, conf=0.25, imgsz=1080,  verbose=False)
    for idx, result in enumerate(results):
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        obb = result.obb  # Oriented boxes object for OBB outputs

        if len(boxes.cls) == 0:
            continue
        
        xywh = boxes.xyxy.data.cpu().numpy().round()
        cls = boxes.cls.data.cpu().numpy().round()
        conf = boxes.conf.data.cpu().numpy()
        for i, (ci, xy, confi) in enumerate(zip(cls, xywh, conf)):
            submit_json.append(
                {
                    'frame_id': idx,
                    'event_id': i+1,
                    'category': category_labels[int(ci)],
                    'bbox': list([int(x) for x in xy]),
                    "confidence": float(confi)
                }
            )

    with open('./result/' + path.split('\\')[-1][:-4] + '.json', 'w', encoding='utf-8') as up:
        json.dump(submit_json, up, indent=4, ensure_ascii=False)

WARNING ⚠️ imgsz=[1080] must be multiple of max stride 32, updating to [1088]

WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

WARNING ⚠️ imgsz=[1080] must be multiple of max stride 32, updating to [1088]

WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generat

In [ ]:
!\rm result/.ipynb_checkpoints/ -rf
!\rm result.zip
!zip -r result.zip result/

'\rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'\rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


  adding: result/ (260 bytes security) (stored 0%)
  adding: result/test_1.json (172 bytes security) (deflated 92%)
  adding: result/test_10.json (172 bytes security) (deflated 93%)
  adding: result/test_11.json (172 bytes security) (deflated 91%)
  adding: result/test_12.json (172 bytes security) (deflated 93%)
  adding: result/test_13.json (172 bytes security) (deflated 93%)
  adding: result/test_14.json (172 bytes security) (deflated 91%)
  adding: result/test_15.json (172 bytes security) (deflated 91%)
  adding: result/test_16.json (172 bytes security) (deflated 92%)
  adding: result/test_17.json (172 bytes security) (deflated 92%)
  adding: result/test_18.json (172 bytes security) (deflated 92%)
  adding: result/test_19.json (172 bytes security) (deflated 92%)
  adding: result/test_2.json (172 bytes security) (deflated 91%)
  adding: result/test_20.json (172 bytes security) (deflated 92%)
  adding: result/test_21.json (172 bytes security) (deflated 92%)
  adding: result/test_22.js